In [ ]:
# @title Setup (Deps)
try:
  from google.colab import drive, userdata
  if not os.path.exists("owl2vec_star"):
    !rm -rf * .config
    !git clone https://github.com/mmtftr/owl2vec_star.git .

  import os

  wandb_api_key = userdata.get('WANDB_API_KEY')
  drive.mount('/content/drive')
  os.environ['WANDB_API_KEY'] = wandb_api_key
  if not os.path.exists("/content/drive/MyDrive/research/GO_emb"):
    raise Exception("GO_emb directory not found in drive")

  !cp -r /content/drive/MyDrive/research/GO_emb/* .
  !unzip go-basic.zip -d go-basic
  !unzip go-full.zip -d go-full
  %pip install -r requirements_owl2vec.txt
  %cd mmtf-eval
except:
  device = "mps" if torch.mps.is_available() else "cpu"

In [1]:

# @title Configuration
base_ontology = "go-basic" #@param ["go-basic", "go-full"]
embedding_types = "biobert,owl2vec" #@param {type:"string"}
models = "mlp,svm" #@param {type:"string"}
input_type = "concatenate" #@param ["concatenate", "minus"]
random_seed = 42 #@param {type:"integer"}

emb_types_str = ", ".join([f"'{emb_type}'" for emb_type in embedding_types.split(',')])
models_str = ", ".join([f"'{model}'" for model in models.split(',')])

config = f"""
# Project settings
project_name: "go-embedding-evaluation"
base_ontology: {base_ontology}
embedding_types: [{emb_types_str}]
models: [{models_str}]
input_type: {input_type}
random_seed: {random_seed}


# Training parameters
batch_size: 64
learning_rate: 0.001
num_epochs: 100
"""

# with open("config.yaml", "w") as f:
#   f.write(config)
config


'\n# Project settings\nproject_name: "go-embedding-evaluation"\nbase_ontology: go-basic\nembedding_types: [\'biobert\', \'owl2vec\']\nmodels: [\'mlp\', \'svm\']\ninput_type: concatenate\nrandom_seed: 42\n\n\n# Training parameters\nbatch_size: 128\nlearning_rate: 0.001\nnum_epochs: 100\n'

In [ ]:
!python embedding_evaluation.py